#1. **Environment Setup**

In [ ]:
# Install the necessary libraries

In [1]:
!pip install transformers torch accelerate

In [ ]:
# Login to Hugging Face:

from huggingface_hub import login

# Enter your Hugging Face access token here
token = "hf_jNmAPohKaQMfqddfXYEzef"
login(token)

#2. **Loading Model & Tokenizer**

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

# Load the model with memory-efficient settings
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", torch_dtype='auto', device_map='auto')

# Explanation:
# - AutoTokenizer loads the correct tokenizer for the model.
# - AutoModelForCausalLM loads the model for generating text.
# - 'torch_dtype' and 'device_map' help in using GPU efficiently.

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

#3. **Add Guardrail System Prompt**

In [4]:
SYSTEM_PROMPT = """
You are a helpful assistant.

Rules you must follow:
- If you are unsure, say: "I’m not certain about that."
- Do NOT invent facts, statistics, sources, or links.
- Do NOT guess answers.
- Ask clarification questions when needed.
- Prefer accuracy over creativity.
- Keep responses concise and factual.
"""

#4. **Conversation Memory**

In [5]:
conversation_history = []

#5. **Guardrail Validation Function**

In [6]:
def guardrail_check(text):

    risky_phrases = [
        "studies prove",
        "guaranteed",
        "always true",
        "100% certain",
        "scientifically proven"
    ]

    for phrase in risky_phrases:
        if phrase.lower() in text.lower():
            return (
                "I may not be fully certain about that information. "
                "Please verify from reliable sources."
            )

    return text

#6. **Conversation Chatbot Logic**

In [7]:
def chat_with_bot(user_input):

    # Add user input
    conversation_history.append(f"User: {user_input}")

    # Build prompt with system rules
    prompt = (
        SYSTEM_PROMPT
        + "\n"
        + "\n".join(conversation_history)
        + "\nAssistant:"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,

        # 🔐 Anti-hallucination settings
        max_new_tokens=150,
        temperature=0.2,        # LOWER randomness
        top_p=0.9,
        do_sample=True,
        repetition_penalty=1.2,
        eos_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_reply = response.split("Assistant:")[-1].strip()

    # Apply guardrail validation
    assistant_reply = guardrail_check(assistant_reply)

    conversation_history.append(f"Assistant: {assistant_reply}")

    return assistant_reply

#4. **Code Requirements-Make it Conversational**

In [9]:
# Running the Chatbot:

print("Welcome to the AI Chatbot!")

while True:
    user_input = input("User: ")

    if user_input.lower() == "quit":
        print("Goodbye!")
        break

    reply = chat_with_bot(user_input)
    print(f"Bot: {reply}")

Welcome to the AI Chatbot!
User: hi
Bot: Hello! Is there something you would like to know or discuss? Please let me know how I can assist you.
User: How is weather today in Kuwait?
Bot: I'm just an AI model, so I don't have access to real-time information on the current weather conditions in Kuwait. However, I can suggest some reliable websites where you can find up-to-date weather forecasts for your desired location. Would you like me to do that?
User: quit
Goodbye!
